In [ ]:
import pandas as pd
import torch
torch.manual_seed(0)
result_df = pd.read_csv('workflow.csv')
result_df

In [ ]:
app_names = []
for index, row in result_df.iterrows():
    app_names.append(row["ActiveWindow"])
app_names

In [ ]:
INPUT_FOLDER = "./workflow/input/"
OUTPUT_FOLDER = "./workflow/output"

from run_workflow_open import run_batch_inference
full_jsons = run_batch_inference(INPUT_FOLDER+"*", app_names, OUTPUT_FOLDER)
# full_jsons

In [ ]:
def find_smallest_bbox(entries, mousepos_x, mousepos_y):
    smallest_entry = None
    smallest_area = float('inf')

    for entry in entries:
        x1, y1, x2, y2 = entry["bbox"]['x1'], entry["bbox"]['y1'], entry["bbox"]['x2'], entry["bbox"]['y2']

        # Calculate the area of the current bounding box
        area = (x2 - x1) * (y2 - y1)

        if x1 <= mousepos_x <= x2 and y1 <= mousepos_y <= y2:
            # The mouse position is within the current bounding box
            if area < smallest_area:
                smallest_area = area
                smallest_entry = entry

    return smallest_entry

In [ ]:
new_event_log = result_df.copy()
new_event_log

In [ ]:
import os
from pathlib import Path
import shutil
import cv2
import matplotlib.pyplot as plt
import glob
import json 

TEMPLATE_FOLDER_PATH = "dropbox_closed_templates"
dirpath = Path(TEMPLATE_FOLDER_PATH)
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)
os.makedirs(TEMPLATE_FOLDER_PATH, exist_ok=True)

# Create a list to store template information in JSON lines format
template_list = []

for index, row in result_df.iterrows():
    print(index)
    print(row["ScreenshotId"])
    full_json = full_jsons[row["ScreenshotId"]]
    # result_df.loc[index,'enabled_activities'] = ";".join(full_json["activity_names"])
    img_resize_shape = full_json["img_resize_shape"]  # type: ignore
    activity_names = full_json["activity_names"]  # type: ignore

    screenshot_id = row["ScreenshotId"]

    img = cv2.imread(INPUT_FOLDER + "/" + screenshot_id)
    img_shape = img.shape

    resize_factor = img_resize_shape[0] / img_shape[0]

    img = cv2.resize(img, (int(img_resize_shape[1]), int(img_resize_shape[0])))
    img_copy = img.copy()
    mousepos_x = int(row['MousePositionX'] * resize_factor)
    mousepos_y = int(row['MousePositionY'] * resize_factor)

    img = cv2.circle(img, (mousepos_x, mousepos_y), radius=5, color=(0, 0, 255), thickness=5)
    # cv2.imshow(str(index), cv2.circle(img, (mousepos_x, mousepos_y), radius=5, color=(0, 0, 255), thickness=5))
    # cv2.waitKey()

    clicked_compo_info = find_smallest_bbox(activity_names, mousepos_x, mousepos_y)
    print(clicked_compo_info)
    bbox = clicked_compo_info["bbox"]
    cropped_img = img_copy[bbox['y1']-5:bbox['y2']+5, bbox['x1']-5:bbox['x2']+5]
    
    # Save the cropped image in the "closed_templates" folder
    template_name = "cropped"
    template_file_name = f"{template_name}_{index}.png"
    template_path = os.path.join(TEMPLATE_FOLDER_PATH, template_file_name)
    cv2.imwrite(template_path, cropped_img)
    
    # Create JSON line entry and add it to the list
    json_line_entry = {
        "template_name": template_file_name,
        "activity_name": clicked_compo_info["activity_name"],
        "template_file_name": template_file_name,
        "threshold": 0.98
    }
    template_list.append(json_line_entry)

# Save the JSON lines file in the "closed_templates" folder
json_lines_file = os.path.join(TEMPLATE_FOLDER_PATH, "metadata.jsonl")
with open(json_lines_file, "w") as f:
    for item in template_list:
        f.write(json.dumps(item) + "\n")

In [ ]:
from run_workflow_closed import run_batch_inference

OUTPUT_FOLDER = "./workflow_closed_output_without_bbox"
dirpath = Path(OUTPUT_FOLDER)
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

full_jsons = run_batch_inference(INPUT_FOLDER+"*", app_names, OUTPUT_FOLDER, TEMPLATE_FOLDER_PATH)

In [ ]:
new_event_log = result_df.copy()
new_event_log

In [ ]:
import cv2
import matplotlib.pyplot as plt
import glob


for index, row in result_df.iterrows():
    print(index)
    print(row["ScreenshotId"])
    full_json = full_jsons[row["ScreenshotId"]]
    img_resize_shape = full_json["img_resize_shape"]
    activity_names = full_json["activity_names"]  # type: ignore
    
    screenshot_id = row["ScreenshotId"]
    
    img = cv2.imread(INPUT_FOLDER + "/" + screenshot_id)
    img_shape = img.shape

    resize_factor = img_resize_shape[0] / img_shape[0]

    img = cv2.resize(img, (int(img_resize_shape[1]), int(img_resize_shape[0])))

    mousepos_x = int(row['MousePositionX'] * resize_factor)
    mousepos_y = int(row['MousePositionY'] * resize_factor)

    img = cv2.circle(img, (mousepos_x, mousepos_y), radius=7, color=(0, 0, 255), thickness=10)
    # cv2.imshow("image", img)
    # cv2.waitKey(0)
    smallest_entry = find_smallest_bbox(activity_names, mousepos_x, mousepos_y)
    activity_names_set = set()
    print(smallest_entry)
    if smallest_entry:
        if not smallest_entry["activity_name"]:
            smallest_entry["activity_name"] = "df"
        print(smallest_entry["activity_name"])
        # cv2.imwrite(str(index) + "_" + screenshot_id + "_"+ smallest_entry["activity_name"] + ".jpg", img)
        new_event_log.loc[index, 'Activity'] = smallest_entry["activity_name"]
        activity_names_set.add(smallest_entry["activity_name"])
    for activity in full_json["activity_names"]:
        print(activity["is_vism"])
        if activity["is_vism"]:
            activity_names_set.add(activity["activity_name"])
    new_event_log.loc[index, 'Enabled Activities'] = "; ".join(activity_names_set)

In [ ]:
new_event_log

In [ ]:
# new_event_log.to_csv("closed_translucent_workflow_small_without_bbox.csv", index=False)

In [ ]:
import pm4py

# log = pm4py.read_('<path-to-xes-log-file.xes>')
dataframe = pm4py.format_dataframe(new_event_log, case_id='CaseId', activity_key='Activity', timestamp_key='TimestampLocal')
event_log = pm4py.convert_to_event_log(dataframe)
process_model = pm4py.discover_bpmn_inductive(event_log)
pm4py.view_bpmn(process_model)